In [242]:
import numpy as np
from scipy import linalg as lg

# ALS

In [332]:
def ALS(tensor, dimensions, rank):
    len_dim = len(dimensions)

    factors = np.array([np.random.rand(dimensions[i], rank) for i in range(len_dim)], dtype=object)

    eps = 1.0
    it = 0

    while eps > 10**(-14) and it < 100:

        eps = 0.0

        for i in range(len_dim):

            t = np.arange(i)
            t = np.append(t, np.arange(i+1, len_dim))
            t = np.append(t, i)

            sweep_l = np.ones(rank).reshape(1, -1)

            for j in range(len_dim):
                if j != i:
                    sweep_l = lg.khatri_rao(sweep_l, factors[j])

            sweep_r = np.transpose(tensor, t).reshape(-1, dimensions[i])

            old_factor = factors[i]

            factors[i] = np.linalg.lstsq(sweep_l, sweep_r, rcond=None)[0].T

            eps = max(eps, np.linalg.norm(old_factor - factors[i]) / np.linalg.norm(old_factor))

        it += 1

    return factors, it

# Create Tensor

In [336]:
sizes = np.array((10, 10, 20))
T = np.zeros(sizes)
for i in range(sizes[0]):
    for j in range(sizes[1]):
        for k in range(sizes[2]):
            T[i, j, k] = i + j + k

# Run ALS

In [341]:
rank = 10
factors, it = ALS(T, sizes, rank)
print(it)

100


In [342]:
len_dim = len(sizes)
sweep_l = np.ones(rank).reshape(1, -1)

for j in range(len_dim):
    sweep_l = lg.khatri_rao(sweep_l, factors[j])

print(np.linalg.norm(np.sum(sweep_l, axis=1) - T.reshape(-1,)) / np.linalg.norm(T.reshape(-1,)))

1.0372205245016462e-15
